# 魔方自动还原

原理步骤：
  - 图像识别(openCV)
  - 操控电脑(pyautogui)
  - 魔方还原(Kociemba)
  - 群论工具(group.py)

In [1]:
cd ../src

e:\BaiduSyncdisk\python\rubik_cube.py-master\src


In [2]:
from rubik import template, Cube, scale_match
from tools import expand_cube

## 一行代码

In [6]:
Cube(interval=0.100000001).auto_solve_cube(wait=False)

魔方识别完毕
还原需要 21 步
魔方已还原，请检查


["R'",
 "D'",
 'B',
 'L',
 'U',
 'D',
 'B',
 'D2',
 'R',
 "L'",
 "F'",
 'B2',
 'U2',
 'R2',
 'L2',
 "U'",
 'L2',
 'F2',
 'D2',
 'B2',
 'U2']

## 图像检测，定位

In [5]:
# 初始化
cube = Cube(interval=0.12)

In [7]:
# 显示识别结果
cube.show_dectition()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'image' in function 'cvDestroyWindow'


In [8]:
# 实际识别过程
## Canny 化 => 从小到大放缩匹配
image = cube.image
_, Loc, ratio = scale_match(image, template, show=True)

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'Template' in function 'cvDestroyWindow'


In [ ]:
# 放缩匹配代码：scale_match.py 
# 图像允许一定范围内的放缩，必要时可修改参数

## 计算位置

In [9]:
# 检查鼠标位置是否正确
cube.check_facets(sides=[0]) 

正在检查顶面，注意查看鼠标位置是否准确


In [10]:
# 获取魔方分布
state = cube.get_cube_distribution(string_code=True)
# 显示六个面的信息
print(expand_cube(state))


          |*********|
          |*L**L**L*|
          |*********|
          |*D**U**L*|
          |*********|
          |*U**F**L*|
          |*********|
 *********|*********|*********|*********
 *L**L**B*|*L**L**L*|*L**L**L*|*L**F**L*
 *********|*********|*********|*********
 *L**L**L*|*U**F**L*|*L**R**F*|*R**B**L*
 *********|*********|*********|*********
 *L**L**L*|*L**L**L*|*L**B**B*|*R**D**L*
 *********|*********|*********|*********
          |*********|
          |*L**L**L*|
          |*********|
          |*L**D**R*|
          |*********|
          |*L**B**U*|
          |*********|



## 操控电脑

In [11]:
# 检验基本旋转 -- pyautogui
cube.check_basic_moves(facets="U")

当前正在旋转的面为 U


In [12]:
# 换心公式
cube.shift_center()

In [13]:
# 自动还原
cube.auto_solve_cube(wait=False)

魔方识别完毕


ValueError: Error. Probably cubestring is invalid

In [14]:
# 设置低延时
cube.interval = 0.100000001
cube.auto_solve_cube(wait=False)

魔方识别完毕


ValueError: Error. Probably cubestring is invalid

## 构造给定图案

In [15]:
# 演示效果
state = "DUDUUUDUDBRBRRRBRBLFLFFFLFLUDUDDDUDUFLFLLLFLFRBRBBBRBR"
cube.to_cube_state(state)

ValueError: Error. Probably cubestring is invalid

In [16]:
f * g

NameError: name 'f' is not defined

### 关于魔方群
1. **广义魔方群**：
    $$
    (C_2^{12}\rtimes S_{12}) \times (C_3^{8}\rtimes S_{8})
    $$
 
    注：(棱块群) 直积 (角块群)

2. **魔方群**：六个基本旋转生成，广义魔方群的真子群。

    注：实际上只用四个旋转就能生成魔方群了
   
3. 上帝之数 = 20

In [20]:
from math import factorial
2**12 * factorial(12) * 3 ** 8 * factorial(8)

519024039293878272000

### 构造方法

1. Kociemba 算法（2-phase algorithm）

2. 广义魔方群 <=> 魔方状态
   $$
   \begin{align*}
   Magic\ Group\ G&\rightarrow\ \{states\ of \ Rubik's\ Cube\}\\
   f&\mapsto f(I)
   \end{align*}
   $$

3. 魔方还原，本质上是求 状态 $S$ 的可逆元 $f$，并将 $f$ 写成基本旋转的组合。
   $$
   S \Rightarrow f(S) = I \Rightarrow f\cdot S = I
   $$

4. 借助群作用，扩展 Kociemba 算法的应用范围
   $$
   \begin{align*}
   &f_1\cdot S_{current} = I\\
   &f_2\cdot S_{target} = I\\
   &\Rightarrow S_{target} = f_2^{-1} = f_2^{-1}\cdot f_1\cdot S_{current}\\
   &\Rightarrow f_2^{-1}\cdot f_1 S_{mix} = I,\ where\ S_{mix} = f_1^{-1}\cdot f_2
   \end{align*}
   $$

简言之：将状态 $S_{current}\Rightarrow S_{target}$ 转化为普通的魔方还原问题 $S_{mix} \Rightarrow I$.